In [53]:
from __future__ import print_function
from __future__ import division
import os.path as op
import os
import pandas as pd
from collections import defaultdict
from itertools import groupby

In [49]:
evaldict = defaultdict(lambda: 1)
blast_dict = {}

In [50]:
with open("../data/clust_reps_vs_tara.translated.out") as infile:
    for i, l in enumerate(infile):
        if i<10:
            print(l)
        number = i
        ev = float(l.split("\t")[-2])
        npc = l.split("\t")[0]
        if ev < evaldict[npc]:
            evaldict[npc] = ev
            blast_dict[npc] = l.split("\t")[1:]

pvp1011	OM-RGC.v1.031721166	91.5	94	8	0	1	94	1	94	5.6e-43	179.9

pvp1008	OM-RGC.v1.018020304	49.7	181	84	3	29	202	3	183	1.1e-41	176.8

pvp1008	OM-RGC.v1.022691721	47.4	175	78	3	29	202	3	164	7.3e-35	154.1

pvp1008	OM-RGC.v1.019063349	45.6	180	92	1	28	201	1	180	9.6e-35	153.7

pvp1008	OM-RGC.v1.020581820	44.8	174	90	3	35	202	1	174	1.6e-34	152.9

pvp1008	OM-RGC.v1.017849209	43.6	181	95	3	29	202	3	183	8.1e-34	150.6

pvp1008	OM-RGC.v1.015895412	41.1	192	99	3	22	202	6	194	8.1e-34	150.6

pvp1008	OM-RGC.v1.019596046	44.6	177	96	2	28	202	1	177	1.1e-33	150.2

pvp1008	OM-RGC.v1.019437873	44.5	173	95	1	29	201	6	177	1.8e-33	149.4

pvp1008	OM-RGC.v1.019426250	42.5	179	96	3	29	200	3	181	4.4e-32	144.8



In [70]:
len_dict = {}

def read_fasta(file_handle):
    '''Fasta iterator'''
    for header, group in groupby(file_handle, lambda line: line[0] == '>'):
        if header:
            line = next(group)
            name = line[1:].strip()
        else:
            seq = ''.join(line.strip() for line in group)
            yield name, seq
            
for name, seq in read_fasta(open("../data/clust_reps.fasta")):
    len_dict[name]=len(seq)

In [67]:
close_match=0
for b in blast_dict:
    aln_len = int(blast_dict[b][2])
    q_len = len_dict[b]
    if aln_len/q_len > 0.75 and float(blast_dict[b][1])>35:
        close_match += 1

In [68]:
print(close_match)

911


In [10]:
cnames = "queryId, subjectId, percIdentity, alnLength, mismatchCount, gapOpenCount, queryStart, queryEnd, subjectStart, subjectEnd, eVal, bitScore".split(",")
df = pd.read_csv("../data/clust_reps_vs_tara.translated.out", sep="\t", names=cnames)
df.columns

Index([u'queryId', u' subjectId', u' percIdentity', u' alnLength',
       u' mismatchCount', u' gapOpenCount', u' queryStart', u' queryEnd',
       u' subjectStart', u' subjectEnd', u' eVal', u' bitScore'],
      dtype='object')

In [25]:
total_matches = len(df.queryId.unique())

In [21]:
!cp /Users/jmb/Desktop/ViralFate/OrthoMCL/phage_v_phage/groups.txt ../data/
!cp /Users/jmb/Desktop/ViralFate/OrthoMCL/phage_v_phage/goodProteins.fasta ../data/

In [72]:
with open("../data/groups.txt") as infile:
    for i, l in enumerate(infile):
        groupnum = i 
print("out of {i} omcl groups, {j} have matches to OM-RGC database, "
      "{k} matched at greater than 35% identity over 75% of the protein".format(i=groupnum, j=total_matches, k=close_match))

out of 2805 omcl groups, 1232 have matches to OM-RGC database, 911 matched at greater than 35% identity over 75% of the protein
